In [74]:
import pandas as pd
import fbprophet as ph

In [75]:
cal_edi = pd.read_csv('./interpolated_edi.csv')

In [76]:
cal_edi.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [77]:
cal_edi

country        EDI       index
0            Brazil   4.678294  2020-02-01
1            Brazil   4.653117  2020-02-02
2            Brazil   4.627939  2020-02-03
3            Brazil   4.602762  2020-02-04
4            Brazil   4.577584  2020-02-05
...             ...        ...         ...
1363  Untied States  10.175226  2020-06-27
1364  Untied States  10.178105  2020-06-28
1365  Untied States  10.180984  2020-06-29
1366  Untied States  10.183863  2020-06-30
1367  Untied States  10.186742  2020-07-01

[1368 rows x 3 columns]

In [78]:
edis = pd.read_csv('./EDI_indicators.csv')

In [79]:
edis.replace({'UnitedKingdom':'United Kingdom',
                        'UnitedStates':'Untied States',
                        'SouthAfrica':'South Africa'}, inplace = True)
edis = edis.loc[edis['country'] != 'HongKong', ]

In [80]:
edis

country       index        EDI
0            Brazil  2008-01-01   2.375691
1            Brazil  2008-02-01   2.556433
2            Brazil  2008-03-01   2.623345
3            Brazil  2008-04-01   2.859230
4            Brazil  2008-05-01   3.352902
...             ...         ...        ...
1505  Untied States  2020-03-01  10.195001
1506  Untied States  2020-04-01   8.744228
1507  Untied States  2020-05-01   9.243445
1508  Untied States  2020-06-01  10.100372
1509  Untied States  2020-07-01  10.186742

[1359 rows x 3 columns]

In [81]:
timeindex_2010_2020 = pd.date_range(start = '1/1/2010', end = '2/14/2020')
timeindex_0215_0701 = pd.date_range(start = '2/15/2020', end = '7/1/2020')
prediction_interval = len(pd.date_range(start = '2/14/2020', end = '7/1/2020'))

In [82]:
timeindex_2010_2020

DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08',
               '2010-01-09', '2010-01-10',
               ...
               '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08',
               '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12',
               '2020-02-13', '2020-02-14'],
              dtype='datetime64[ns]', length=3697, freq='D')

In [83]:
def forecast_economic_indicator(df, f_interval, freq = 'D'):
    # get dataframe column names
    df_names = df.columns
    # change dataframe column names
    df.columns = ['ds', 'y']
    # convert ds column name
    df['ds'] = pd.to_datetime(df['ds'])
    
    # initiate a prophet model
    m = ph.Prophet()
    # fit on dataframe
    m.fit(df)
    # define future prediction
    future = m.make_future_dataframe(periods = f_interval, freq = freq)
    # make prediction
    forecast = m.predict(future)
    # predict historical values
    hist_predict = forecast[['ds','yhat']][:-f_interval]
    # predict future values
    future_predict = forecast[['ds','yhat']][-f_interval:]
    
    return df, hist_predict, future_predict, df_names

In [84]:
edi_diff = pd.DataFrame(columns=['index', 'EDI', 'yhat', 'country'])

In [85]:
for country in edis.country.unique():
    
    temp_edi = edis.loc[edis['country'] == country, ]
    temp_edi.set_index('index', inplace = True)
    temp_edi.index = pd.to_datetime(temp_edi.index)
    temp_edi = temp_edi.reindex(timeindex_2010_2020)
    temp_edi = temp_edi.interpolate().fillna(method = 'ffill')
    temp_edi = temp_edi.reset_index()
    temp_edi.drop(['country'], axis = 1, inplace = True)
    
    temp_cal_edi = cal_edi.loc[cal_edi['country'] == country, ]
    temp_cal_edi.set_index('index', inplace = True)
    temp_cal_edi.index = pd.to_datetime(temp_cal_edi.index)
    temp_cal_edi = temp_cal_edi.reindex(timeindex_0215_0701)
    temp_cal_edi = temp_cal_edi.reset_index()
    temp_cal_edi.drop(['country'], axis = 1, inplace = True)
    
    _, _, predicted_edi, _ = forecast_economic_indicator(temp_edi, prediction_interval)
    
    combined = temp_cal_edi.merge(predicted_edi, left_on = 'index', right_on = 'ds', how = 'left')
    combined['country'] = country
    
    edi_diff = edi_diff.append(combined)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [86]:
edi_diff['response'] = edi_diff['yhat'] - edi_diff['EDI']

In [69]:
edi_diff.drop(['EDI', 'yhat', 'ds'], axis = 1, inplace = True)

In [87]:
edi_diff[]

index        EDI       yhat        country         ds  response
0   2020-02-15   4.325809   4.909036         Brazil 2020-02-15  0.583227
1   2020-02-16   4.300631   4.912205         Brazil 2020-02-16  0.611573
2   2020-02-17   4.275454   4.915322         Brazil 2020-02-17  0.639868
3   2020-02-18   4.250276   4.918377         Brazil 2020-02-18  0.668101
4   2020-02-19   4.225099   4.921313         Brazil 2020-02-19  0.696215
..         ...        ...        ...            ...        ...       ...
133 2020-06-27  10.175226  12.320775  Untied States 2020-06-27  2.145549
134 2020-06-28  10.178105  12.324533  Untied States 2020-06-28  2.146428
135 2020-06-29  10.180984  12.327755  Untied States 2020-06-29  2.146771
136 2020-06-30  10.183863  12.330399  Untied States 2020-06-30  2.146536
137 2020-07-01  10.186742  12.332591  Untied States 2020-07-01  2.145849

[1242 rows x 6 columns]

In [71]:
edi_diff.to_csv('response.csv')

In [53]:
predicted_edi

ds      yhat
3697 2020-02-15  4.909036
3698 2020-02-16  4.912205
3699 2020-02-17  4.915322
3700 2020-02-18  4.918377
3701 2020-02-19  4.921313
...         ...       ...
3831 2020-06-28  4.979398
3832 2020-06-29  4.981681
3833 2020-06-30  4.983670
3834 2020-07-01  4.985326
3835 2020-07-02  4.986699

[139 rows x 2 columns]

In [91]:
new_prediction_interval = len(pd.date_range(start = '7/2/2020', end = '7/1/2021'))

In [90]:
edis_US = edis.loc[edis['country'] == 'Untied States', :]
edis_US.drop(['country'], axis = 1, inplace = True),
edis_US

index        EDI
1359  2008-01-01   4.322160
1360  2008-02-01   4.304472
1361  2008-03-01   4.208769
1362  2008-04-01   4.421110
1363  2008-05-01   4.401690
...          ...        ...
1505  2020-03-01  10.195001
1506  2020-04-01   8.744228
1507  2020-05-01   9.243445
1508  2020-06-01  10.100372
1509  2020-07-01  10.186742

[151 rows x 2 columns]

In [92]:
_,_,pre, _ = forecast_economic_indicator(edis_US, new_prediction_interval, freq = 'D')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [93]:
pre

ds       yhat
151 2020-07-02  11.138574
152 2020-07-03  10.693584
153 2020-07-04  10.225053
154 2020-07-05   9.741087
155 2020-07-06   9.250484
..         ...        ...
511 2021-06-27  13.722781
512 2021-06-28  13.494763
513 2021-06-29  13.217657
514 2021-06-30  12.893769
515 2021-07-01  12.526675

[365 rows x 2 columns]